In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import platform
import matplotlib as mpl
import random
from copy import copy
import re

# if platform.system() == 'Darwin':
#     matplotlib.use("TkAgg")

import matplotlib.animation
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy as sp
import pprint

from pathlib import Path

from relnet.utils.general_utils import *
from relnet.agent.baseline.baseline_agent import *
from relnet.agent.mcts.mcts_agent import *

from relnet.evaluation.storage import EvaluationStorage
from relnet.evaluation.experiment_conditions import *
from relnet.evaluation.file_paths import FilePaths
from relnet.evaluation.experiment_conditions import *
from relnet.visualization import *


algorithm_class = 'planning'

full_g_sizes = [25, 50, 75, 100, 125, 150, 175, 200]
g_sizes = [25, 50, 75, 100, 125, 150, 175, 200]
# g_sizes = [100]
exp_ids = [f'timingsv2_kh_{g_size}' for g_size in g_sizes]
exp_ids



storage = EvaluationStorage()
fp = FilePaths('/experiment_data', 'aggregate', setup_directories=True)


considered_agents_fig = [GreedyAgent.algorithm_name,
                         CostSensitiveGreedyAgent.algorithm_name,
                         StandardMCTSAgent.algorithm_name,
                         SGUCTAgent.algorithm_name
                     ]

considered_agents_table = [
                     RandomAgent.algorithm_name,
                     
                     GreedyAgent.algorithm_name,
                     CostSensitiveGreedyAgent.algorithm_name,
                     
                           
                     LowestDegreeProductAgent.algorithm_name,
                     FiedlerVectorAgent.algorithm_name,
                     EffectiveResistanceAgent.algorithm_name,
                     MinCostAgent.algorithm_name,
                     LBHBAgent.algorithm_name,
                           
                     StandardMCTSAgent.algorithm_name,
                     SGUCTAgent.algorithm_name,
                     ]


algo_display_names = {
                     GreedyAgent.algorithm_name: "Greedy",
                     CostSensitiveGreedyAgent.algorithm_name: "GreedyCS",
                     StandardMCTSAgent.algorithm_name: "UCT",
                     SGUCTAgent.algorithm_name: "SG-UCT",
                     
                     RandomAgent.algorithm_name: "Random",
                     LowestDegreeProductAgent.algorithm_name: "LDP",
                     FiedlerVectorAgent.algorithm_name: "FV",
                     EffectiveResistanceAgent.algorithm_name: "ERes",
                     MinCostAgent.algorithm_name: "MinCost",
                     LBHBAgent.algorithm_name: "LBHB",

                     }

In [ ]:
def fetch_timings(exp_ids, which_agents):
    all_timings = []
    for i in range(len(exp_ids)):
        g_size = g_sizes[i]
        exp_id = exp_ids[i]
        
        
        fp_in = FilePaths('/experiment_data', exp_id, setup_directories=False)
        for f in fp_in.timings_dir.iterdir():
            if f.is_file():
                header_names = ['timestep', 'before_or_after', 'g_list_idx', 'timestamp', 'num_sims']
                timings_df = pd.read_csv(f, sep=',', header=None, names=header_names)
                
                if f.name.startswith("."):
                    continue
                
                fname_parts = str(f.name).split("-")
                algo = fname_parts[0]
                obj_fun = fname_parts[1]
                network_generator = fname_parts[2]
                if 'rw' in exp_id:
                    g_seed = fname_parts[3]
                else:
                    g_seed = int(fname_parts[3].split("_")[-1])
                
                if algo not in which_agents:
                    continue
                
                timings_entry = {}
                timings_entry['algo'] = algo_display_names[algo]
                timings_entry['obj_fun'] = obj_fun
                timings_entry['network_generator'] = network_generator
                timings_entry['g_seed'] = g_seed
                timings_entry['g_size'] = g_size if 'rw' not in exp_id else -1
                
                total_ms = (timings_df[['timestamp']].diff().dropna().sum()).iloc[0]
                total_sims = (timings_df[['num_sims']].dropna().sum()).iloc[0]
                
                num_moves = len(timings_df) / 2
                
                timings_entry['total_ms'] = total_ms
                timings_entry['total_s'] = total_ms / 1000
                timings_entry['total_sims'] = total_sims
                
                timings_entry['num_moves'] = num_moves
                timings_entry['avg_s_per_move'] = total_ms / num_moves / 1000
                timings_entry['avg_sims_per_move'] = total_sims / num_moves

                all_timings.append(timings_entry)
        

    agg_df = pd.DataFrame(all_timings)
    agg_df = agg_df.sort_values(by=["algo"], ascending=[True], axis=0)
    return agg_df

In [ ]:
timings_df = fetch_timings(exp_ids, considered_agents_fig)

In [ ]:
print(timings_df)

In [ ]:
# average of SG-UCT in hours for graphs of size 200
relevant_entries = timings_df[(timings_df["g_size"] == 200) & (timings_df["algo"] == "SG-UCT")]
relevant_entries["total_s"].mean() / 3600

In [ ]:
obj_funs = ["global_eff", "lcs_targeted"]
obj_funs_display ={"global_eff": "$\mathcal{F}_{E}$", "lcs_targeted" :"$\mathcal{F}_{R}$"}

metrics = ["total_s", "avg_s_per_move", "total_sims", "avg_sims_per_move"]
metrics_display = ["Total seconds", "Mean seconds per step", "Total $\mathcal{F}$ evaluations", "Mean $\mathcal{F}$ evaluations per step"]
metrics_display_dict = {metrics[i]: metrics_display[i] for i in range(len(metrics))}


sns.set(font_scale=8.5)
plt.rc('font', family='serif')
mpl.rcParams['text.usetex'] = True
mpl.rcParams["lines.linewidth"] = 16
mpl.rcParams["lines.markersize"] = 25
dims = (2 * 8.26 * len(metrics), 2.25 * 8.26 * len(obj_funs))
legend_i, legend_j = 1, 3

fig, axes = plt.subplots(len(obj_funs), len(metrics), figsize=dims, squeeze=False, sharey=False)



for i, obj_fun in enumerate(obj_funs):
    for j, metric in enumerate(metrics):
        ax = axes[i][j]

        ax.set(yscale="log")
        plot_df = timings_df[timings_df["obj_fun"] == obj_fun]
        plot_df = plot_df.rename(columns={"g_size": "$N$", metric: metrics_display[j], "algo": "Algorithm"})
        sns.lineplot(data=plot_df, x="$N$", y=metrics_display[j], hue="Algorithm", ax=ax)
        ax.set_ylabel('')
        
        if i == 0:
            ax.set_xlabel('')

        ax.tick_params(axis='x', which='major', labelsize=68)
        ax.tick_params(axis='y', which='major', labelsize=80)
        #         ax.tick_params(axis='both', which='minor', labelsize=8)
        ax.set_xticks(full_g_sizes)
        
        ax.legend_.remove()
        handles, labels = ax.get_legend_handles_labels()
        if i == legend_i and j == legend_j:
            legend_ax = ax
        
legend_ax.legend(handles[1:], labels[1:], loc='lower right', borderaxespad=0.1, fontsize="small")


pad = 12.5
cols = metrics
for ax, col in zip(axes[0], cols):
    ax.annotate(f"{metrics_display_dict[col]}",
                xy=(0.5, 1), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size='small', ha='center', va='baseline')

rows = obj_funs
for ax, row in zip(axes[:, 0], rows):
    ax.annotate(f"{obj_funs_display[row]}", xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
                rotation=90,
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center',
               annotation_clip=False)


fig.tight_layout(pad=0.5, w_pad=0.1, h_pad=0.5)
fig.savefig(fp.figures_dir / "timings_all.pdf")
# fig.tight_layout()




In [ ]:
import datetime
import numpy as np

def format_timing(val_ms):
    print(val_ms)
    if np.isnan(float(val_ms)):
        return "---"
    
    val_seconds = int(val_ms / 1000)
    if val_seconds == 0:
        timing_string = "<00:01"
    else:
        timing_string = '{:01}:{:02}:{:02}'.format(val_seconds//3600, val_seconds%3600//60, val_seconds%60)
    return timing_string

rw_timings = fetch_timings(["timingsv2_rw"], considered_agents_table)
rw_timings['timings_string'] = rw_timings['total_ms'].apply(format_timing)

In [ ]:
rw_timings.head(10)

In [ ]:
rwt_pivot = pd.pivot_table(rw_timings, values='timings_string',
                   index= ['obj_fun', 'network_generator', 'g_seed'],
                   columns=['algo'],
                   aggfunc='first')

rwt_pivot = rwt_pivot[[algo_display_names[a] for a in considered_agents_table]]

rwt_pivot

In [ ]:
def clean_and_write_latex(pivot_table, cols_order, filename):
    latex_df = pivot_table.copy()
    texfile =  str(fp.figures_dir / filename)
    fh = open(texfile, 'w')
    table_colformat = f"ccc|{''.join(['r'] * len(cols_order)) }"
    latex_df.to_latex(buf=fh, float_format="{:0.3f}".format, column_format=table_colformat)
    fh.close()

    replace_dict = {
        r"reduction\\_policy": r"$\phi$",
        r"sim\\_policy\\_bias": r"$\\beta$",
        r"exp\\_name": r"Experiment",
        r"synth\\_25": r"KH-25",
        r"synth\\_50": r"KH-50",
        r"synth\\_75": r"KH-75",
        r"rw\\_internet\\_topology": r"Internet",
        r"rw\\_metro": r"Metro",        
        r"rw": "Real-world",
        
        r"C\\_p": r"$C_p$",
        
        r"SG-UCT": "SG-UCT (ours)",
        r"internet\\_topology": "Internet",
        r"metro": "Metro",
        r"graph\\_id": "Graph",
        r"g\\_seed": "Graph",
        
        r"nan±nan": r"---",
        r"NaN": r"---",
        r"nan": r"---",
        r"obj\\_fun": r"$\\mathcal{F}$",
        r"objective\\_function": r"$\\mathcal{F}$",
        r"network\\_size" : r"$|V|$",
        r"network\\_generator" : r"$\\mathbf{G}$",
        r"kaiser\\_hilgetag": r"KH",
        r"internet\\_topology": r"Internet",
        r"metro": r"Metro",
        r"agent": r"Agent",
        r"algorithm" : r"",
        r"algo" : r"",
        r"\\toprule": r"",
        r"global\\_eff": r"$\\mathcal{F}_{E}$",
        r"lcs\\_targeted": r"$\\mathcal{F}_{R}$",
        r"±(\d+\.\d+)": r"\\tiny{$\\pm\g<1>$}",
        r"±---": r"\\tiny{$\\pm0.000$}"
    }

    with open(texfile, 'r') as f:
        raw_content = f.read()

    processed_content = raw_content
    for orig, targ in replace_dict.items():
        processed_content = re.sub(orig, targ, processed_content, flags = re.M)

    with open(texfile, 'w') as g:
        g.write(processed_content)

In [ ]:
clean_and_write_latex(rwt_pivot, considered_agents_table, "rw_timings.tex")